In [1]:
from prettyprinter import pprint
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
## add src folder to path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

# print paths 
print("Current working directory:", os.getcwd())
print("Python executable:", sys.executable)
pprint(sys.version, width=80, indent=4)
print("Python path:", sys.path) 

Current working directory: d:\SYNEREPOS\rag-research-agent-fork\notebooks
Python executable: d:\SYNEREPOS\.venv\Scripts\python.exe
'3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 '
'bit (AMD64)]'
Python path: ['C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313', 'd:\\SYNEREPOS\\.venv', '', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages', '__editable__.retrieval_graph-0.0.1.finder.__path_hook__', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages\\win32', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages\\win32\\lib', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages\\Pythonwin', 'd:\\SYNEREPOS\\rag-research-agent-fork\\src']


Create a faiss index for testing 

In [14]:
from langchain_openai import AzureOpenAIEmbeddings
model = "text-embedding-ada-002" 
openai_api_version = os.environ["OPENAI_API_VERSION"]

embedding_function = AzureOpenAIEmbeddings(
    openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint = os.environ["AZURE_EMBEDDINGS_ENDPOINT"],
    model = model,
    chunk_size = 16,
    max_retries = 3,
    show_progress_bar = True,
    )

In [15]:
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.vectorstores import DistanceStrategy
from langchain_community.docstore import InMemoryDocstore

index = faiss.IndexFlatIP(1536)

vstore = FAISS(
    index=index,
    embedding_function=embedding_function,
    docstore = InMemoryDocstore(), 
    index_to_docstore_id = {}, 
    distance_strategy = "DistanceStrategy.MAX_INNER_PRODUCT", 
)

# load json file data 
import json
with open("sample_docs.json", "r") as f:
    data = json.load(f)

from langchain_core.documents import Document
documents  = [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in data]

vstore.add_documents(documents)
vstore.save_local("faiss_index")

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:01<00:00,  1.44it/s]


In [16]:
# from IPython.display import Image, display

# display(Image(mygraph.get_graph(xray=True).draw_mermaid_png()))


Use langgraph server (optional)

In [18]:
# https://langchain-ai.github.io/langgraph/tutorials/langgraph-platform/local-server/#launch-langgraph-server

In [19]:
# from langgraph_sdk import get_client

# client = get_client(url="http://localhost:2024")

# async for chunk in client.runs.stream(
#     None,  # Threadless run
#     "agent", # Name of assistant. Defined in langgraph.json.
#     input={
#         "messages": [{
#             "role": "human",
#             "content": "What is LangGraph?",
#         }],
#     },
#     stream_mode="updates",
# ):
#     print(f"Receiving new event of type: {chunk.event}...")
#     print(chunk.data)
#     print("\n\n") 